In [ ]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import os
import pandas as pd
import random
import csv



gtf = "c_elegans.PRJNA13758.WS258.canonical_geneset_plu_NO_inside_genes_same_strand.gtf"



In [11]:
my_col = ["chr","source","type","start","end","dot","strand","dot2","gene_id"]
df_gtf = pd.read_csv(gtf, sep="\t",header=None,names=my_col, comment="#", dtype={"chr" : str, "source" : str,"type" : str, "start" : int,"end" : int,"dot" : str,"strand" : str, "dot2" : str,"gene_id" : str})
df_gtf = df_gtf[~df_gtf["chr"].str.contains("\.") ]    # Take out patches
df_gtf = df_gtf[df_gtf["chr"].str.contains("IV") ]    # Take out patches
df_gtf = df_gtf[df_gtf.type.str.contains("gene")]      # Keep everything that says gene

df_gtf["length"] = df_gtf["end"] - df_gtf["start"]
df_gtf = df_gtf.dropna()
chr_names=df_gtf['chr'].unique().tolist()

print(df_gtf.head())


    chr    source  type  start    end dot strand dot2  \
136  IV  WormBase  gene    695  14926   .      +    .   
176  IV  WormBase  gene   8765  11070   .      -    .   
190  IV  WormBase  gene  15499  20899   .      -    .   
218  IV  WormBase  gene  21493  23198   .      +    .   
236  IV  WormBase  gene  24683  26388   .      -    .   

                                               gene_id  length  
136  gene_id "WBGene00021406"; gene_source "WormBas...   14231  
176  gene_id "WBGene00021407"; gene_source "WormBas...    2305  
190  gene_id "WBGene00021408"; gene_source "WormBas...    5400  
218  gene_id "WBGene00021405"; gene_source "WormBas...    1705  
236  gene_id "WBGene00021409"; gene_source "WormBas...    1705  


In [12]:
df_gtf = df_gtf.sort_values("start")
df_gtf["next_gene_start"] = df_gtf["start"].shift(-1)
df_gtf["next_gene_start-end"] = df_gtf["next_gene_start"] - df_gtf["end"]
df_gtf_3kb = df_gtf[df_gtf["next_gene_start-end"]>5000].copy() #Make sure over 5000 in intergenic space


df_gtf_3kb["random_intergenic_start"] = df_gtf_3kb["start"] + 1000 + random.randint(1,2000)  #Add 1000bp buffer, draw random 1-200
df_gtf_3kb["random_intergenic_end_500"] = df_gtf_3kb["random_intergenic_start"] + 500
df_gtf_3kb["random_intergenic_end_1000"] = df_gtf_3kb["random_intergenic_start"] + 1000
df_gtf_3kb["random_intergenic_end_1500"] = df_gtf_3kb["random_intergenic_start"] + 1500
print(df_gtf_3kb.head())

df_gtf_3kb = df_gtf_3kb.head(1000)

     chr    source  type   start     end dot strand dot2  \
393   IV  WormBase  gene   41471   41620   .      +    .   
428   IV  WormBase  gene   66982   67052   .      +    .   
443   IV  WormBase  gene   80152   80203   .      +    .   
1384  IV  WormBase  gene  146729  146845   .      +    .   
1439  IV  WormBase  gene  165077  169980   .      +    .   

                                                gene_id  length  \
393   gene_id "WBGene00195502"; gene_source "WormBas...     149   
428   gene_id "WBGene00023131"; gene_source "WormBas...      70   
443   gene_id "WBGene00201447"; gene_source "WormBas...      51   
1384  gene_id "WBGene00197291"; gene_source "WormBas...     116   
1439  gene_id "WBGene00021396"; gene_source "WormBas...    4903   

      next_gene_start  next_gene_start-end  random_intergenic_start  \
393           53401.0              11781.0                    42474   
428           76370.0               9318.0                    67985   
443           86890.0  

In [18]:
def append_fasta_to_df(fa,name,chr,strand,start,end):
    fa = SeqIO.parse(fa, "fasta")
    print("working on gene", name)
    #print(name)
    #print(chr)
    for record in fa:
        
        if record.id == chr:
            seq = (record.seq)[start:end]
            if strand == "-":
                seq = seq.reverse_complement()
            return seq
        
            #print(seq)     
          

In [20]:

def addSequences(df):
    df['fasta_random_intergenic_500'] = df.apply(lambda row: append_fasta_to_df(fa, row['gene_id'], row['chr'], row['strand'], row["random_intergenic_start"],row["random_intergenic_end_500"]), axis=1)
    df['fasta_random_intergenic_1000'] = df.apply(lambda row: append_fasta_to_df(fa, row['gene_id'], row['chr'], row['strand'], row["random_intergenic_start"],row["random_intergenic_end_1000"]), axis=1)
    df['fasta_random_intergenic_1500'] = df.apply(lambda row: append_fasta_to_df(fa, row['gene_id'], row['chr'], row['strand'], row["random_intergenic_start"],row["random_intergenic_end_1500"]), axis=1)
    return df

df_gtf_3kb=addSequences(df_gtf_3kb)    
    

working on gene gene_id "WBGene00195502"; gene_source "WormBase"; gene_biotype "ncRNA";


working on gene gene_id "WBGene00023131"; gene_source "WormBase"; gene_biotype "tRNA";


working on gene gene_id "WBGene00201447"; gene_source "WormBase"; gene_biotype "ncRNA";


working on gene gene_id "WBGene00197291"; gene_source "WormBase"; gene_biotype "ncRNA";


working on gene gene_id "WBGene00021396"; gene_source "WormBase"; gene_biotype "protein_coding";


...

working on gene gene_id "WBGene00045321"; gene_source "WormBase"; gene_biotype "pseudogene";


working on gene gene_id "WBGene00195757"; gene_source "WormBase"; gene_biotype "ncRNA";


working on gene gene_id "WBGene00048150"; gene_source "WormBase"; gene_biotype "piRNA";


working on gene gene_id "WBGene00165955"; gene_source "WormBase"; gene_biotype "piRNA";


working on gene gene_id "WBGene00011374"; gene_source "WormBase"; gene_biotype "protein_coding";


In [52]:

def split_string(s):
    s = s.split(';')[0]
    return s

#df_gtf_3kb["gene_id"] = df_gtf_3kb["gene_id"].astype(str)
#df_gtf_3kb["gene_id"] = df_gtf_3kb["gene_id"].replace("gene_id","")
df_gtf_3kb["gene_id"] = df_gtf_3kb["gene_id"].apply(split_string)
print(df_gtf_3kb.head())

     chr    source  type   start     end dot strand dot2  \
393   IV  WormBase  gene   41471   41620   .      +    .   
428   IV  WormBase  gene   66982   67052   .      +    .   
443   IV  WormBase  gene   80152   80203   .      +    .   
1384  IV  WormBase  gene  146729  146845   .      +    .   
1439  IV  WormBase  gene  165077  169980   .      +    .   

                       gene_id  length  next_gene_start  next_gene_start-end  \
393   gene_id "WBGene00195502"     149          53401.0              11781.0   
428   gene_id "WBGene00023131"      70          76370.0               9318.0   
443   gene_id "WBGene00201447"      51          86890.0               6687.0   
1384  gene_id "WBGene00197291"     116         152656.0               5811.0   
1439  gene_id "WBGene00021396"    4903         175725.0               5745.0   

      random_intergenic_start  random_intergenic_end_500  \
393                     42474                      42974   
428                     67985         

In [55]:






            
def writeFasta(df,name_out,length):
    length = str(length)
    with open(name_out,"w") as outfile:
        x = 0
        for row in range(len(df)):
            
            outfile.write(">random_"+str(x) + "\n" + str(df['fasta_random_intergenic_'+length].iloc[row]) + "\n")
            #outfile.write(">"+df["gene_id_name"].iloc[row] + "\n" + df['fasta_'+length+'_upstream'].iloc[row] + "\n")
            x = x + 1
    
                

In [56]:
    

writeFasta(df_gtf_3kb,"Random_Intergenic_500.fa",500)
writeFasta(df_gtf_3kb,"Random_Intergenic_1000.fa",1000)
writeFasta(df_gtf_3kb,"Random_Intergenic_1500.fa",1500)
